Este patrón de Unión es muy interesante por que nos va a permitir meter al map por stream varios archivos
o tablas, y unirlos a una tabla que previamente hemos cargado en memoria. Es decir, al contrario que hacíamos en los patrones de unión interno, por la izquierda, derecha, etc. que metíamos las dos tablas que se iban a unir, aquí una tabla la metemos en memoria y la otra u otras la metemos por el stream para unirlas en el Mapper. No será necesario barajar ni ordenar ni reducir. Solo desarrollamos mapper_init, una función a la que llamaremos desde el init y será la que cargará el archivo con la tabla en memoria, y despues en el Map, haremos la unión por la entrada del stream. Aquí vamos a tener la dificultad del tamaño de la memoria, es la que nos pondrá límites. 

In [ ]:
%config IPCompleter.greedy=True

VERSIÓN 1 - Esta primera versión abre el archivo del sistema de archivos, que está guardado en un directorio.
Lo hace, pero no del todo bien porque:
    1 - Hay código que no está dentro de ningún método. Tengo que meterlo en una función o método, esto queda feo y poco profesional, pero...
    2 - ...si lo meto en una función que sea llamada por ejemplo, desde el mapper_init, no reconoce el archivo csv y aparece un error.
    El punto 2 se lo he consultado a Agustín, a ver si me da un poco de norte.
 EDITO: He conseguido cargar un fichero y meterlo en un diccionario en la versión 5


In [15]:
%%writefile unionReplicadaV1.py
#!/usr/bin/env python

from mrjob.job import MRJob
import re,sys,os


class unionReplicadaV1(MRJob):
  
    diccionario={}
    archivo = open("clientes.csv")
    for linea in archivo:
        encontrado=re.search('[a-zA-Z]',linea[0])#Para que no tenga en cuenta las cabeceras de las tablas
        if encontrado==None:
            datos = linea.split(";")
            diccionario[datos[0]]=datos

    def mapper_init(self):
        #Llamamos a cargaTabla
        self.tablaA=self.diccionario #tablaA será el diccionario que le viene desde cargaTabla"""
    def mapper(self,_,line):
        self.linea=line.split(';')
        encontrado=re.search('[a-zA-Z]',self.linea[0])#Para que no tenga en cuenta las cabeceras de las tablas
        if encontrado==None: #¡¡¡CUIDADO CON LOS ÁMBITOS DE LAS VARIABLES!!!!
            self.clave=self.linea[6] #Extraigo la clave del fichero de entrada 
            if self.clave in self.tablaA: #Si encuentra la clave en el diccionario unimos y sacamos
                yield self.clave,(self.linea,self.tablaA.get(self.clave) )
            else:#Si no está, sacamos la línea del archivo pero sin su complementario del diccionario de memoria
                yield self.clave,(self.linea,"null")
    """def mapper_final(self):
         # A continuación se comprueba que la clave del fichero de entrada esté en el diccionario
        if self.clave in self.tablaA: #Si encuentra la clave en el diccionario unimos y sacamos
            yield self.clave,(self.linea,self.tablaA.get(self.clave) )
        else:#Si no está, sacamos la línea del archivo pero sin su complementario del diccionario de memoria
            yield self.clave,(self.linea,"null")"""
                    
if __name__ == '__main__':
    unionReplicadaV1.run()

Overwriting unionReplicadaV1.py


In [16]:
!python unionReplicadaV1.py ventas.csv > unionReplicadaV1.txt

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory C:\Users\MRSANC~1\AppData\Local\Temp\unionReplicadaV1.mrsanchez.20200915.101904.332691
Running step 1 of 1...
job output is in C:\Users\MRSANC~1\AppData\Local\Temp\unionReplicadaV1.mrsanchez.20200915.101904.332691\output
Streaming final output from C:\Users\MRSANC~1\AppData\Local\Temp\unionReplicadaV1.mrsanchez.20200915.101904.332691\output...
Removing temp directory C:\Users\MRSANC~1\AppData\Local\Temp\unionReplicadaV1.mrsanchez.20200915.101904.332691...


VERSIÓN 2 - Dado los problemas para cargar en el entorno un archivo para meterlo en la memoria, he optado por crear los datos directamente en el mapper_init, para poder desarrollar el algoritmo que permita hacer la unión desde memoria con el archivo/os de entrada. El resultado es el código debajo. Dejo pendiente mejorar este patrón para cargar desde la caché los archivos o lo que sea que me tiene frito, sin necesidad de escribir los datos del diccionario a mano; lo que se trata es que el diccionario (hashmap en Java) se cree automáticamente cogiendo los datos desde la DistributedCache de Hadoop (creo que es así).

In [ ]:
%%writefile unionReplicadaV2.py
#!/usr/bin/env python

from mrjob.job import MRJob
import pandas as pd
import re,sys,os
"""Versión 2 - En el mapper init cargo los datos directamente"""
class unionReplicadaV2(MRJob):
    """Esta versión es solo para comprobar el algoritmo de búsqueda y unión bases de datos pequeñas. No es nada funcional. Solo pruebas.
    Como patrón, el algoritmo de búsqueda que es el que vamos a usar en todas las versiones Unión replicada."""
    def mapper_init(self):
        #Cargamos los datos en memoria
        self.datos = {'3':[3,3738,"New York,NY"],
        '4':[4,12946,"New York,NY"],
        '5':[5,17556,"San Diego, CA"],
        '9':[9,3443,'Oakland, CA']}
        
    def mapper(self,_,line):
        linea=line.split(';')
        encontrado=re.search('[a-zA-Z]',linea[0])#Para que no tenga en cuenta las cabeceras de las tablas
        if encontrado==None:
            clave=linea[0] #Extraigo la clave del fichero de entrada 
            # A continuación se comprueba que la clave del fichero de entrada esté en el diccionario
            if clave in self.datos: #Si encuentra la clave en el diccionario unimos y sacamos
                yield clave,(linea,self.datos.get(clave) )
            else:#Si no está, sacamos la línea del archivo pero sin su complementario del diccionario de memoria
                yield clave,(linea,"null")
                    
if __name__ == '__main__':
    unionReplicadaV2.run()

In [ ]:
!python unionReplicadaV2.py tablaB.csv > unionReplicadaV2.txt --quiet

VERSIÓN 3 No funciona - Usamos la libreía MRStep para la carga, configuración y ejecución por pasos.
Son tres pasos:
    Paso 1 - inicio_estructura() Creación de la estructura Diccionario donde iran los datos en memoria del primer fichero que se le pasa
    Paso 2 - carga_datos() Se llena la estructura con los datos del primer fichero
    Paso 3 - replicated_join() Se aplica el algoritmo de búsqueda y unión.

In [ ]:
%%writefile unionReplicadaV3.py
#!/usr/bin/env python

from mrjob.job import MRJob
from mrjob.step import MRStep
import re,os

class unionReplicadaV3(MRJob):

    
    def inicio_estructura(self):
        #Diccionario para cargar los datos en memoria
        self.diccionario = {}
            
    def carga_datos(self,_,line):
        for linea in line:
            encontrado=re.search('[a-zA-Z]',linea[0])#Para que no tenga en cuenta las cabeceras de las tablas
            if encontrado==None:
                datos = linea.split(";")
                self.diccionario[datos[0]]=datos
    
    def replicated_join(self,input_path):
        linea=line.split(';')
        encontrado=re.search('[a-zA-Z]',linea[0])#Para que no tenga en cuenta las cabeceras de las tablas
        if encontrado==None:
            clave=linea[0] #Extraigo la clave del fichero de entrada 
            # A continuación se comprueba que la clave del fichero de entrada esté en el diccionario
            if clave in self.datos: #Si encuentra la clave en el diccionario unimos y sacamos
                yield clave,(linea,self.diccionario.get(clave) )
            else:#Si no está, sacamos la línea del archivo pero sin su complementario del diccionario de memoria
                yield clave,(linea,"null")
    
    def steps(self):
        return [MRStep(mapper_init=self.inicio_estructura,
                        mapper=self.carga_datos),
                MRStep(mapper_raw=self.replicated_join)]
                    
if __name__ == '__main__':
    unionReplicadaV3.run()

In [ ]:
!python unionReplicadaV3.py > unionReplicadaV3.txt

Versión 4 No funciona -  Se ha desarrollado con algunas líneas de https://github.com/manursanchez/mrjob/blob/master/mrjob/examples/mr_most_used_word.py


In [ ]:
%%writefile unionReplicadaV4.py
#!/usr/bin/env python

from mrjob.job import MRJob
import re

class unionReplicadaV4(MRJob):
    FILES=['tablaA.csv']
    
    def configure_args(self):
        super(unionReplicadaV4, self).configure_args()

        # allow for alternate stop words file
        self.add_file_arg(
            '--tablaA-file',
            dest='tablaA_file', #ERROR 
            default=None,
            help='Tabla que cargaremos en memoria',
        )
        
    def cargaTabla(self):
        self.diccionario={} 
        for linea in self.tablaA_file:
            encontrado=re.search('[a-zA-Z]',linea[0])#Para que no tenga en cuenta las cabeceras de la tabla
            if encontrado==None:
                datos = linea.split(";")
                diccionario[datos[0]]=datos

        return diccionario    
   
    def mapper_init(self):
        
        #Llamamos a cargaTabla
        self.tablaA=self.cargaTabla()
        
    def mapper(self,_,line):
        self.linea=line.split(';')
        encontrado=re.search('[a-zA-Z]',self.linea[0])#Para que no tenga en cuenta las cabeceras de las tablas
        if encontrado==None:
            self.clave=self.linea[6] #Extraigo la clave del fichero de entrada 
           
    def mapper_final(self):
         # A continuación se comprueba que la clave del fichero de entrada esté en el diccionario
        if self.clave in self.tablaA: #Si encuentra la clave en el diccionario unimos y sacamos
            yield self.clave,(self.linea,self.tablaA.get(self.clave) )
        else:#Si no está, sacamos la línea del archivo pero sin su complementario del diccionario de memoria
            yield self.clave,(self.linea,"null")
                    
if __name__ == '__main__':
    unionReplicadaV4.run()

In [ ]:
!python unionReplicadaV4.py tablaB.csv > unionReplicadaV4.txt

Pequeño pgramita para rellenar un diccionario clave valor a partir de un archivo

In [ ]:
%%writefile llenarDiccionario.py
#!/usr/bin/env python
import re
diccionario={}
archivo = open("tablaA.csv")
for linea in archivo:
    encontrado=re.search('[a-zA-Z]',linea[0])#Para que no tenga en cuenta las cabeceras de las tablas
    if encontrado==None:
        datos = linea.split(";")
        diccionario[datos[0]]=datos

print (diccionario)

In [ ]:
!python llenarDiccionario.py

FUNCIONA. VERSIÓN DEFINITIVA.

Versión 5 - Combinando con la Versión 4, he conseguido cargar el archivo como argumento., lo que me permite crear el diccionario en memoria y realizar la Unión replicada. Funciona pero solo me devuelve un bloque con la unión replicada del proceso Map.
EDITO 15/09/2020 El maldito error de que solo me devolviera un conjunto de registros era por que el algoritmo lo aplicaba fuera del ámbito del primer if (el que comprueba la expresión regular). Vamos, un error de primero de carrera. :-(


In [16]:
%%writefile unionReplicadaV5.py
#!/usr/bin/python

import re
from mrjob.job import MRJob

def cargarFicheroEnMemoria(self,fichero):
    diccionario={}
    with open(fichero) as f:
        self.tablaEnMemoria = set(line.strip() for line in f)
    for linea in self.tablaEnMemoria:
        #Para que no tenga en cuenta las cabeceras de las tablas
        encontrado=re.search('[a-zA-Z]',linea[0])
        if encontrado==None:
            datos = linea.split(";")
            diccionario[datos[0]]=datos
    return diccionario

class unionReplicadaV5(MRJob):
    FILES = ['archivos_datos/tiendas.csv']
    fichero='tiendas.csv'    
    def mapper_init(self):
        #Nos devuelve la estructura diccionario rellena con los datos del fichero
        self.dicTablaA=cargarFicheroEnMemoria(self,self.fichero) 
       
    def mapper(self,_,line):
        self.linea=line.split(';')
        encontrado=re.search('[a-zA-Z]',self.linea[0])#Para que no tenga en cuenta las cabeceras de las tablas
        if encontrado==None:
            #Aplicamos algoritmo: 
            self.clave=self.linea[1] #Se extrae la clave del fichero que entra por línea de comandos 
             # Se comprueba que la clave del fichero de entrada esté en el diccionario
            if self.clave in self.dicTablaA: #Si encuentra la clave en el diccionario unimos y cedemos (yield) resultado
                yield self.clave,(self.linea,self.dicTablaA.get(self.clave) )
            else:#Si no está, sacamos la línea del archivo pero sin su complementario del diccionario de memoria. Uniòn por la izquierda.
                yield self.clave,(self.linea,"null")
                
if __name__ == '__main__':
    unionReplicadaV5.run()

Overwriting unionReplicadaV5.py


In [18]:
!python unionReplicadaV5.py archivos_datos/articulos_stock.csv archivos_datos/articulos_stock2.csv > unionReplicadaV5.txt

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory C:\Users\cyber\AppData\Local\Temp\unionReplicadaV5.manuel.20201205.194147.851299
Running step 1 of 1...
job output is in C:\Users\cyber\AppData\Local\Temp\unionReplicadaV5.manuel.20201205.194147.851299\output
Streaming final output from C:\Users\cyber\AppData\Local\Temp\unionReplicadaV5.manuel.20201205.194147.851299\output...
Removing temp directory C:\Users\cyber\AppData\Local\Temp\unionReplicadaV5.manuel.20201205.194147.851299...


Versión 6 funciona - Utilizando herramientas de test y creando un objeto MRJob intento pasarle el diccionario ya relleno al constructor de la clase unionReplicada V6, me aparece un error de argumentos, si lo hago con un archivo funciona, si le mando un diccionario da error. EDITO 15/09/2020 --> Ya que me funciona el algoritmo de la versión 5, voy a desarrollar esta versión de test, que me parece muy interesante sobre todo cuando quiero hacer uniones replicadas de varios archivos usando solo una estructura en memoria; puedo crear un ejemplo claro de ejecución de varios runner. EDITO 17/09/2020 --> Consigo que el algoritmo funcione mediante llamada al objeto MRJob.

In [1]:
%%writefile testUnionReplicadaV6.py
import unittest
import re,json
from unionReplicadaV6 import unionReplicadaV6

class TestReplicatedJoin(unittest.TestCase):
        
    def setUp(self):
        #Archivo para la unión
        self.archivo = "archivos_datos/tablaB.csv"
        
        """#self.archivo="clientes.csv"
        self.archivo = "ventas.csv"
        self.diccionario={}
        with open(self.archivo) as f:
            self.tablaEnMemoria = set(line.strip() for line in f)
        for linea in self.tablaEnMemoria:
            encontrado=re.search('[a-zA-Z]',linea[0])#Para que no tenga en cuenta las cabeceras de las tablas
            if encontrado==None:
                datos = linea.split(";")
                self.diccionario[datos[0]]=datos"""
        
        #print ("Diccionario relleno: ",self.diccionario) #Veo si el diccionario está relleno
        
    def test_rj(self):
        #self.archivo2=json.dumps(self.diccionario)
        mr_job = unionReplicadaV6(['--runner=inline',self.archivo])#Mando al objeto 
                                                                   #MRJOB el diccionario.
                                                                    #ERROR. Si pongo un
                                                                    #archivo, sí funciona.
        with mr_job.make_runner() as runner: #Ejecuto el MRJob
            runner.run()
            for key,value in mr_job.parse_output(runner.cat_output()):
                print (key,value) 
            
if __name__ == '__main__':
    unittest.main()

Writing testUnionReplicadaV6.py


In [2]:
%%writefile unionReplicadaV6.py
#!/usr/bin/python

import re
from mrjob.job import MRJob
# Método para cargar el fichero en memoria
def cargarFicheroEnMemoria(self,fichero):
    diccionario={}
    with open(fichero) as f:
        self.tablaEnMemoria = set(line.strip() for line in f)
    for linea in self.tablaEnMemoria:
        #Para que no tenga en cuenta las cabeceras de las tablas
        encontrado=re.search('[a-zA-Z]',linea[0])
        if encontrado==None:
            datos = linea.split(";")
            diccionario[datos[0]]=datos
    return diccionario
class unionReplicadaV6(MRJob):
    
    #archivo para cargarlo en memoria en forma de diccionario
    FILES = ['archivos_datos\\tablaA.csv']
    fichero='tablaA.csv'    
    def mapper_init(self):
        #Nos devuelve la estructura diccionario rellena con los datos del fichero
        self.dicTablaA=cargarFicheroEnMemoria(self,self.fichero)
    
    def mapper(self,_,line):
        self.linea=line.split(';')
        encontrado=re.search('[a-zA-Z]',self.linea[0])#Para que no tenga en cuenta las cabeceras de las tablas
        if encontrado==None:
            #Aplicamos algoritmo: 
            self.clave=self.linea[0] #Se extrae la clave del fichero que llega desde el RUNNER
             # Se comprueba que la clave del fichero de entrada esté en el diccionario
            if self.clave in self.dicTablaA: #Si encuentra la clave en el diccionario unimos y cedemos (yield) resultado
                yield self.clave,(self.linea,self.dicTablaA.get(self.clave) )
            else:#Si no está, sacamos la línea del archivo pero sin su complementario del diccionario de memoria. Uniòn por la izquierda.
                yield self.clave,(self.linea,"null")
            
if __name__ == '__main__':
    unionReplicadaV6.run()

Writing unionReplicadaV6.py


In [3]:
!python testUnionReplicadaV6.py

3 [['3', '35314', 'Esto es texto del post'], ['3', '3738', 'New York,NY']]
3 [['3', '44002', 'Esto es mas texto aun'], ['3', '3738', 'New York,NY']]
5 [['5', '44921', 'Aqui digo otra cosa'], ['5', '17556', 'San Diego, CA']]
5 [['5', '44920', 'Otra cosa que que se dice'], ['5', '17556', 'San Diego, CA']]
8 [['8', '48675', 'Decimos muchas cosas, es impresionante'], 'null']


FILES: archivos_datos\tablaA.csv will appear in same directory as job script, not a subdirectory
FILES: archivos_datos\tablaA.csv will appear in same directory as job script, not a subdirectory
No configs specified for inline runner
.
----------------------------------------------------------------------
Ran 1 test in 0.210s

OK
